### Setup Environment

In [15]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/paula/Ironhack/Ironhack_Data_Analytics.json"

In [16]:
from google.cloud import bigquery

In [17]:
client = bigquery.Client()

### Step 1: Calculate the royalties of each sales for each author

Write a SELECT query to obtain the following output:

* Title ID
* Author ID
* Royalty of each sale for each author
    * The formula is:
        ```
        sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100
        ```
    * Note that `titles.royalty` and `titleauthor.royaltyper` are divided by 100 respectively because they are percentage numbers instead of floats.

In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sales.

In [18]:
query_1 = """
SELECT
    titles.title_id,
    authors.au_id,
    (titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) AS sales_royalty

FROM
        `ironhack-data-analytics.publications.titles` titles
        
INNER JOIN
        `ironhack-data-analytics.publications.sales` sales
ON
        sales.title_id = titles.title_id
INNER JOIN
        `ironhack-data-analytics.publications.titleauthor` titleauthor 
ON
        titleauthor.title_id = sales.title_id
INNER JOIN
        `ironhack-data-analytics.publications.authors` authors 
ON
        authors.au_id = titleauthor.au_id
ORDER BY
    authors.au_id DESC
"""

In [19]:
query_1 = client.query(query=query_1)
df_1=query_1.to_dataframe()
df_1.head(10)

,title_id,au_id,sales_royalty
0,PS2091,998-72-3567,13.140
1,PS2091,998-72-3567,1.971
2,PS2091,998-72-3567,6.570
3,PS2106,998-72-3567,17.500
4,PS2091,998-72-3567,49.275
5,PS2091,899-46-2035,13.140
6,MC3021,899-46-2035,4.485
7,PS2091,899-46-2035,1.971
8,PS2091,899-46-2035,6.570
9,PS2091,899-46-2035,49.275


### Step 2: Aggregate the total royalties for each title for each author

Using the output from Step 1, write a query to obtain the following output:

* Title ID
* Author ID
* Aggregated royalties of each title for each author
    * Hint: use the *SUM* subquery and group by both `au_id` and `title_id`

In the output of this step, each title should appear only once for each author.

In [20]:
query_2="""
WITH royalties_by_titauth AS(
    SELECT
        titles.title_id,
        authors.au_id,
        (titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) AS sales_royalty

    FROM
            `ironhack-data-analytics.publications.titles` titles

    INNER JOIN
            `ironhack-data-analytics.publications.sales` sales
    ON
            sales.title_id = titles.title_id
    INNER JOIN
            `ironhack-data-analytics.publications.titleauthor` titleauthor 
    ON
            titleauthor.title_id = sales.title_id
    INNER JOIN
            `ironhack-data-analytics.publications.authors` authors 
    ON
            authors.au_id = titleauthor.au_id)
        SELECT
            title_id,
            au_id,
            SUM(sales_royalty) as total_royalties
        FROM
            royalties_by_titauth 
        GROUP BY
            1,2
        ORDER BY
            au_id DESC  
"""

In [21]:
query_2 = client.query(query=query_2)
df_2=query_2.to_dataframe()
df_2.head(10)

,title_id,au_id,total_royalties
0,PS2091,998-72-3567,70.956
1,PS2106,998-72-3567,17.500
2,PS2091,899-46-2035,70.956
3,MC3021,899-46-2035,7.176
4,PC8888,846-92-7186,50.000
5,TC3218,807-91-6654,83.800
6,PS1372,756-30-7391,32.385
7,PS1372,724-80-9391,10.795
8,BU1111,724-80-9391,17.925
9,MC3021,722-51-5454,21.528


### Step 3: Calculate the total profits of each author

Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

* Author ID
* Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.


In [22]:
query_3="""
WITH royalties_totals AS(
SELECT 
    title_id,
    au_id, 
    au_lname, 
    au_fname,
    SUM(advance_au) AS total_advances,
    SUM(sales_royalties) AS total_royalties
FROM 
    (SELECT
        titles.title_id,
        authors.au_id,
        titles.price,
        (titles.advance * titau.royaltyper / 100) AS advance_au,
        titles.royalty,
        sales.qty,
        authors.au_lname,
        authors.au_fname,
        titau.royaltyper,
        (titles.price * sales.qty * titles.royalty / 100 * titau.royaltyper / 100) AS sales_royalties

    FROM
            `ironhack-data-analytics.publications.titles` titles

    INNER JOIN
            `ironhack-data-analytics.publications.sales` sales
    ON
            sales.title_id = titles.title_id
    INNER JOIN
            `ironhack-data-analytics.publications.titleauthor` titau
    ON
            titau.title_id = sales.title_id
    INNER JOIN
            `ironhack-data-analytics.publications.authors` authors 
    ON
            authors.au_id = titau.au_id)
GROUP BY
    1,2,3,4) 
            SELECT
            au_id,
            au_fname,
            au_lname,
            SUM(total_royalties + total_advances) as total_profits
        FROM
            royalties_totals 
        GROUP BY
            1,2,3
        ORDER BY
            total_profits DESC  
        LIMIT 3
"""

In [23]:
query_3 = client.query(query=query_3)
df_3=query_3.to_dataframe()
df_3

,au_id,au_fname,au_lname,total_profits
0,722-51-5454,Michel,DeFrance,22521.528
1,213-46-8915,Marjorie,Green,14162.110
2,899-46-2035,Anne,Ringer,12128.132


## Challenge 2

Elevating from your solution in Challenge 1 , create a table named `most_profiting_authors` to hold the data about the most profiting authors. The table should have 2 columns:

* `au_id` - Author ID
* `profits` - The profits of the author aggregating the advances and royalties

In [26]:
query_4="""
CREATE TABLE `ironhack-data-analytics.publications.most_profiting_authors_paula` AS(
WITH royalties_totals AS(
SELECT 
    title_id,
    au_id, 
    au_lname, 
    au_fname,
    SUM(advance_au) AS total_advances,
    SUM(sales_royalties) AS total_royalties
FROM 
    (SELECT
        titles.title_id,
        authors.au_id,
        titles.price,
        (titles.advance * titau.royaltyper / 100) AS advance_au,
        titles.royalty,
        sales.qty,
        authors.au_lname,
        authors.au_fname,
        titau.royaltyper,
        (titles.price * sales.qty * titles.royalty / 100 * titau.royaltyper / 100) AS sales_royalties

    FROM
            `ironhack-data-analytics.publications.titles` titles

    INNER JOIN
            `ironhack-data-analytics.publications.sales` sales
    ON
            sales.title_id = titles.title_id
    INNER JOIN
            `ironhack-data-analytics.publications.titleauthor` titau
    ON
            titau.title_id = sales.title_id
    INNER JOIN
            `ironhack-data-analytics.publications.authors` authors 
    ON
            authors.au_id = titau.au_id)
GROUP BY
    1,2,3,4) 
            SELECT
            au_id,
            au_fname,
            au_lname,
            SUM(total_royalties + total_advances) as total_profits
        FROM
            royalties_totals 
        GROUP BY
            1,2,3 
        LIMIT 3)
"""

In [27]:
query_4 = client.query(query=query_4)
df_4=query_4.to_dataframe()
df_4

""


In [28]:
query_5="""
SELECT *
FROM `ironhack-data-analytics.publications.most_profiting_authors_paula`
"""

In [29]:
query_5 = client.query(query=query_5)
df_5=query_5.to_dataframe()
df_5

,au_id,au_fname,au_lname,total_profits
0,648-92-1872,Reginald,Blotchet-Halls,4033.460
1,427-17-2319,Ann,Dull,4050.000
2,672-71-3249,Akiko,Yokomoto,3211.992
